In [5]:
import timeit
import math
import os
import sys
import warnings

import numpy as np
from matplotlib import pyplot as plt
from scipy import sparse
from scipy.sparse.linalg import spsolve

from A_matrix import A
from boundary_conditions import boundary_conditions_A, boundary_conditions_b
from setup import *

warnings.filterwarnings('ignore', '.*do not.*', )

recordResults = False
plotOngoingResiudalGraph = False

# How many iterations  ons between plotting residuals
sampleRate = 200

# Time loop
if recordResults:
    os.system("mkdir " + saveDirectory(nx, ny))


start = timeit.default_timer()

# Define momentum loop tolerance:
tolerance = 1e-6

# Only allow one time-step if non transient
# if not transient:

t = np.array([0, 1])

U_new=np.empty(U_new.shape)
U_new.fill(1e-8)

for time in t:

    print("\nTime = " + str(round(time, 1)))

    # Reset iteration counter
    icorr = 1

    # Set number of maximu_m iterations for convergence
    maxcorr = 100000

    # Make directory for timestep
    if recordResults:
        os.system("mkdir " + saveDirectory(nx, ny) + "/" + str(round(time, 2)))

    # Initialise arrays for graphs
    residual_array = np.array([])
    moving_average_array = np.array([])
    moving_average_graph_array = np.array([])

    # Create A matrices:
    A_x = A("x").createMatrix()
    A_x = boundary_conditions_A(A_x, "x")

    A_y = A("y").createMatrix()
    A_y = boundary_conditions_A(A_y, "y")

    # plt.hlines(tolerance, 0, len(moving_average_graph_array)*10, color = "C1", label  = "Tolerance")
    plt.yscale("log")
    plt.xlabel("Iterations")
    plt.ylabel("Residual")

    # Momentum Loop
    while True:

        # Store solution for previous iteration
        U_previous = U_new*1

        # x-equation
        # Create b matrices
        b_x = boundary_conditions_b(b_x, U_previous, U_old, U_old_old, "x")

        # # Solve for u
        A_x = sparse.csr_matrix(A_x)
        u = spsolve(A_x, b_x)

        # y-equation
        # Create b matrices
        b_y = boundary_conditions_b(b_y, U_previous, U_old, U_old_old, "y")

        # # Solve for v
        A_y = sparse.csr_matrix(A_y)
        v = spsolve(A_y, b_y)

        if icorr == 1:
            v1 = v
            u1 = u
            by_1 = b_y
            bx_1 = b_x
            U_previous_1 = U_previous

        if icorr == 2:
            v2 = v
            u2 = u
            bx_2 = b_x
            by_2 = b_y
            U_previous_2 = U_previous


        # if icorr == 6:
            # break
        
        
        U_new = np.vstack((u, v)).T

        # Calculate the residual of each iteration
        SMALL = 1e-16
        normFactor = np.max(U_new) + SMALL
        residual = math.sqrt(np.mean((U_new - U_previous)**2))/normFactor

        # Append residual array with residual
        residual_array = np.append(residual_array, residual)

        # The following is for on-going convergence reports:
        # Print out residual every 100 iterations
        if icorr % 100 == 0:
            moving_average = np.mean(
                residual_array[len(residual_array)-100:len(residual_array)])
            moving_average_array = np.append(
                moving_average_array, moving_average)
            print("Iteration: {:01d},\t Residual = {:.20f},\t normFactor = {:.20f},\t Moving Average = {:.20f},\t Time = {:.5f}".format(
                icorr, residual, normFactor, moving_average, time))

        # Calculate moving average of residual
        if icorr % 10 == 0:
            moving_average_graph = np.mean(
                residual_array[len(residual_array)-10:len(residual_array)])
            moving_average_graph_array = np.append(
                moving_average_graph_array, moving_average_graph)

        # # Print residual progress every 500 iterations
        if icorr % 50 == 0:
            plt.plot(np.arange(100, len(moving_average_graph_array)*10, 10),
                     moving_average_graph_array[10:len(moving_average_graph_array)], label="Moving Average", color="C2")

        if (icorr % 10 == 0) & (icorr > 10):
            plt.plot(np.arange(10, len(moving_average_graph_array)*10, 10),
                     moving_average_graph_array[1:len(moving_average_graph_array)], color="C0",  label="Residual")

        # if icorr <= 10:
        #     plt.plot(np.arange(0, len(residual_array)),
        #              residual_array, color="C0",  label="Residual")
        # plt.plot(icorr, residual, '.', label = "Residual", color = "C0")
        plt.plot(icorr, tolerance, '.', label="Tolerance", color="C1")
        if icorr == 1:
            plt.legend()

        plt.plot(np.arange(0, len(residual_array)), residual_array)


        if plotOngoingResiudalGraph:
            if (icorr % sampleRate == 0):
                plt.draw()
                plt.pause(0.00001)


        # Convergence check
        if residual < tolerance:

            print("\nResiduals have converged:\n")
            print("Iteration: {:01d},\t Residual = {:.20f},\t normFactor = {:.20f},\t Moving Average = {:.20f},\t Time = {:.5f}".format(
                icorr, residual, normFactor, moving_average, time))

            break

        elif icorr > maxcorr:

            break

        # Increment iteration counter
        icorr = icorr + 1

    # Update displacement temporal fields
    U_old_old = U_old
    U_old = U_new

    #------------------#
    # Save displacement field
    if recordResults:
        saveArray(str(round(time, 2)) + "/U_fine_mesh", U_new)

        U_with_boundaries, V_with_boundaries = addBoundaryDisplacements(U_new)
        saveArray(str(round(time, 2)) +
                  "/U_with_boundaries", U_with_boundaries)
        saveArray(str(round(time, 2)) +
                  "/V_with_boundaries", V_with_boundaries)


stop = timeit.default_timer()

print('Run time: ', stop - start)

# sys.stdout.close()

sys.exit()



Time = 0
Iteration: 100,	 Residual = 0.00853734662891287611,	 normFactor = 0.00002436491210464936,	 Moving Average = 26.77136115194027254915,	 Time = 0.00000
Iteration: 200,	 Residual = 0.00381140507665225043,	 normFactor = 0.00004828952280522274,	 Moving Average = 0.09618084006132952835,	 Time = 0.00000
Iteration: 300,	 Residual = 0.00231633300532356305,	 normFactor = 0.00007073359882583674,	 Moving Average = 0.04837251171925616156,	 Time = 0.00000
Iteration: 400,	 Residual = 0.00160128890908036066,	 normFactor = 0.00009116451470605090,	 Moving Average = 0.03085602471497458227,	 Time = 0.00000
Iteration: 500,	 Residual = 0.00118766008064839757,	 normFactor = 0.00010952600411491758,	 Moving Average = 0.02194750535149728524,	 Time = 0.00000
Iteration: 600,	 Residual = 0.00092034201868286276,	 normFactor = 0.00012594325516413098,	 Moving Average = 0.01659335428397766196,	 Time = 0.00000
Iteration: 700,	 Residual = 0.00073464163069060127,	 normFactor = 0.00014059211911687360,	 Moving Ave

SystemExit: 

/home/scottlevie/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
/home/scottlevie/miniconda3/lib/python3.9/site-packages/IPython/core/pylabtools.py:151: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  fig.canvas.print_figure(bytes_io, **kw)


ValueError: Image size of 397x98962 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 432x288 with 1 Axes>